#Fast Fuzzy Matching
This notebook shows how to use TD IDF to both dedupe and match records at scale

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from tqdm import tqdm
from google.colab import drive
import os
from matplotlib import style
style.use('fivethirtyeight')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


The data can be found at the following link:
https://drive.google.com/file/d/1EAXvkiik5EO8FcpEwfX3muQEm6cqPGrQ/view?usp=sharing


In [2]:
names =  pd.read_csv('messy org names.csv',encoding='latin')
print('The shape: %d x %d' % names.shape)
print('There are %d unique values' % names.buyer.shape[0])


The shape: 3651 x 3
There are 3651 unique values


##De duplication:

In [3]:
import re
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

def ngrams(string, n=3):
    string = str(string)
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in "Department":')
print(ngrams('Department'))

     |████████████████████████████████| 64 kB 2.5 MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=8a7ff7461b2d8da1c032d850839a356125ba6f34214374be3ab71e48fd3f4495
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built ftfy
All 3-grams in "Department":
[' De', 'Dep', 'epa', 'par', 'art', 'rtm', 'tme', 'men', 'ent', 'nt ']


In [4]:
import numpy as np
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn #uncomment to install
import sparse_dot_topn.sparse_dot_topn as ct


def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for sparse-dot-topn: filename=sparse_dot_topn-0.3.1-cp37-cp37m-linux_x86_64.whl size=1581136 sha256=1a4de96a69adcc353585215314267d14af99a07b952c1e7cd1f3122595626e77
  Stored in directory: /root/.cache/pip/wheels/3b/3e/02/4ee8cb28ed8b608d530bc43402518a895db8ce89aff8ca4e1f
Successfully built sparse-dot-topn


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

org_names = names['buyer'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(org_names)

In [6]:
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.85)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 0.40517544746398926


#### Comparison to traditional matching
This code prints the time it takes to compare <b>only one</b> item against the population. As you can see, the TD IDF approach can match all items (3,600) significantly faster than it takes to compare a single item using the fuzzywuzzy library.

In [7]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

t1 = time.time()
print(process.extractOne('Ministry of Justice', org_names))
t = time.time()-t1
print("SELFTIMED:", t)
print("Estimated hours to complete for full dataset:", (t*len(org_names))/60/60)

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


('MINISTRY OF JUSTICE', 100)
SELFTIMED: 3.2792763710021973
Estimated hours to complete for full dataset: 3.325732786258062


#### Inputting results into a df:

In [8]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [9]:
matches_df = get_matches_df(matches, org_names, top=1000)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)

,left_side,right_side,similairity
785,Student Loans Company,STUDENT LOANS COMPANY LIMITED,0.915689
539,Cabinet Office : Cabinet Office,Cabinet Office,0.961424
341,Department of Health,Department Health,0.861610
623,University Of Plymouth,Plymouth University,0.851232
585,Forestry Commission,The Forestry Commission,0.904622
855,Derby Homes Ltd,Derby Homes,0.886169
574,University College London Hospitals NHS Foundation Trust,University College of London Hospitals NHS Foundation Trust,0.941518
591,Construction Industry Training Board (CITB),Construction Industry Training Board,0.909534
169,North Yorkshire County Council,North Yorkshire Council,0.902527
649,Stoke-On-Trent City Council,City Of Stoke-on-Trent,0.853927


In [10]:
matches_df.sort_values(['similairity'], ascending=False).head(20)

,left_side,right_side,similairity
637,"MINISTRY OF HOUSING, COMMUNITIES &amp; LOCAL GOVERNMENT","Ministry of Housing, Communities &amp; Local Government : Ministry of Housing, Communities &amp; Local Government",0.985163
57,DEPARTMENT FOR INTERNATIONAL DEVELOPMENT,Department for International Development : Department for International Development,0.984561
234,Department for Work and Pensions,Department for Work and Pensions : Department for Work and Pensions,0.977116
92,DEPARTMENT FOR WORK AND PENSIONS,Department for Work and Pensions : Department for Work and Pensions,0.977116
99,THE UNITED KINGDOM HYDROGRAPHIC OFFICE,United Kingdom Hydrographic Office,0.971264
364,"NHS SOUTH, CENTRAL AND WEST COMMISSIONING SUPPORT UNIT","South, Central and West Commissioning Support Unit",0.969860
45,Department For Transport,Department for Transport : Department for Transport,0.969679
207,HEALTH AND SOCIAL CARE INFORMATION CENTRE,The Health and Social Care Information Centre,0.967980
52,MINISTRY OF JUSTICE,Ministry of Justice : Ministry of Justice,0.967797
68,Ministry of Justice.,Ministry of Justice : Ministry of Justice,0.967797


## Record linkage
Using a similar technique to the above, we can join our messy data to a clean set of master data.

The clean 'Gov Orgs ONS.xlsx' dataset can be found at the following link:
https://drive.google.com/file/d/1uBxlrASNMA215x4o4pu8JmtA2Iu-sLNn/view?usp=sharing

In [ ]:


##################
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re

clean_org_names = pd.read_excel('Gov Orgs ONS.xlsx')
clean_org_names = clean_org_names.iloc[:, 0:6]

org_name_clean = clean_org_names['Institutions'].unique()

print('Vecorizing the data - this could take a few minutes for large datasets...')
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf = vectorizer.fit_transform(org_name_clean)
print('Vecorizing completed...')

from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)

org_column = 'buyer' #column to match against in the messy data
unique_org = set(names[org_column].values) # set used for increased performance


###matching query:
def getNearestN(query):
  queryTFIDF_ = vectorizer.transform(query)
  distances, indices = nbrs.kneighbors(queryTFIDF_)
  return distances, indices

import time
t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(unique_org)
t = time.time()-t1
print("COMPLETED IN:", t)

unique_org = list(unique_org) #need to convert back to a list
print('finding matches...')
matches = []
for i,j in enumerate(indices):
  temp = [round(distances[i][0],2), clean_org_names.values[j][0][0],unique_org[i]]
  matches.append(temp)

print('Building data frame...')  
matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name','Origional name'])
print('Done') 

Vecorizing the data - this could take a few minutes for large datasets...
Vecorizing completed...
getting nearest n...
COMPLETED IN: 0.8935930728912354
finding matches...
Building data frame...
Done


In [ ]:
matches.head(10)

,Match confidence (lower is better),Matched name,Origional name
0,0.96,Offshore Renewable Energy Catapult,Energy Systems Catapult
1,1.01,Derby City Council,University of Derby
2,1.13,Remploy Limited,Sovini Limited
3,0.00,Leicester City Council,Leicester city council
4,0.82,Ministry of Defence,"Ministry of Defence, C&C, C&C"
5,1.10,Thurrock Council,BASILDON AND THURROCK UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
6,0.00,London Borough of Hounslow,London Borough of Hounslow
7,0.00,Northumberland County Council,Northumberland County Council
8,0.80,Scottish Fire and Rescue Service,Derbyshire Fire and Rescue Service
9,1.02,The English Heritage Trust,English Heritage-National Collections: Historic Properties
